In [1]:
import cv2
import numpy as np
import os
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tqdm import tqdm
import csv
import glob
import matplotlib.pyplot as plt

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [2]:
import matplotlib.pyplot as plt
from skimage import measure
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger, EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution3D, MaxPooling3D, ZeroPadding3D
from tensorflow.keras.regularizers import l2

In [3]:
import csv
import shutil
import glob
from subprocess import call
import os.path
from tensorflow.keras.utils import to_categorical
import numpy as np
import tensorflow as tf
import keras
from tensorflow.keras.preprocessing import image as Img
from tqdm import tqdm
import pandas as pd
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
import random
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Flatten, Dense, Concatenate, Dot, Lambda, Input, Dropout, Activation, GlobalAveragePooling3D
from tensorflow.keras.optimizers import Adam, SGD
import matplotlib.pyplot as plt
from tensorflow.keras import backend as K
from keras.regularizers import l2
from keras.layers import Flatten, Average

In [4]:
import tensorflow as tf
np.random.seed(1234)
tf.random.set_seed(1234)
random.seed(1234)

In [5]:
def cp(src_path, dest_path):
    shutil.copy(src_path, dest_path)
    print('done')

In [6]:
cp(r'/kaggle/input/keras-i3d-model/i3d_inception.py', r'/kaggle/working/i3d_inception.py')

done


In [7]:
from i3d_inception import Inception_Inflated3d, _obtain_input_shape, conv3d_bn

In [8]:
# pbar = tqdm(total=len(data.data))
# try:
#     for index, video in enumerate(data.data):
#         path = os.path.join('sequences', video[2] + '-' + 'features')
#         if os.path.isfile(path + '.npy'):
#             pbar.update(1)
#             continue
#         frames = data.get_frames_for_sample(video)
#         frames = data.rescale_list(frames, 40)
        
#         if frames is None:
#             continue
#         sequence = []
#         for frame in frames:
#             img = image.load_img(frame, target_size=(299, 299, 3))
#             x = image.img_to_array(img)
#             sequence.append(x)
#         sequence_array = np.array(sequence)
#         gray_frames = np.array([cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) for frame in sequence_array])
#         # Compute optical flow between consecutive frames
#         flow_frames = []
#         for i in range(1, len(gray_frames)):
#             prev_frame = gray_frames[i - 1]
#             curr_frame = gray_frames[i]
#             flow = cv2.calcOpticalFlowFarneback(prev_frame, curr_frame, None, 0.5, 3, 15, 3, 5, 1.2, 0)
#             flow_frames.append(flow)

#         # Stack the frames along the batch dimension
#         preprocessed_frames = np.expand_dims(np.array(flow_tensor), axis=0)
#         np.save(path + '.npy', preprocessed_frames)
#         pbar.update(1)
# finally:
#     pbar.close()

In [9]:
import tensorflow as tf
np.random.seed(1234)
tf.random.set_seed(1234)
random.seed(1234)

In [10]:
model_rgb = Inception_Inflated3d(include_top=False,
                weights='rgb_kinetics_only',
                input_tensor=None,
                input_shape=(40, 224, 224, 3),
                dropout_prob=0.30,
                endpoint_logit=False,
                classes=400)

model_optical = Inception_Inflated3d(include_top=False,
                weights='flow_kinetics_only',
                input_tensor=None,
                input_shape=(40, 224, 224, 2),
                dropout_prob=0.30,
                endpoint_logit=False,
                classes=400)

49595336/49595336 [==============================] - 0s 0us/step


/kaggle/working/i3d_inception.py:90: UserWarning: This model usually expects 1 or 3 input channels. However, it was passed an input_shape with 2 input channels.
  warnings.warn(


49507528/49507528 [==============================] - 0s 0us/step


In [11]:
conv_model = Model(inputs = model_rgb.input, outputs = model_rgb.output)
optical_model = Model(inputs = model_optical.input, outputs = model_optical.output)


# Rename the layers in each model to ensure uniqueness
for layer in model_rgb.layers:
    layer._name = 'rgb_' + layer.name
for layer in model_optical.layers:
    layer._name = 'optical_' + layer.name

output_rgb = model_rgb.output
output_optical = model_optical.output

# Fusion using element-wise concatenation
concat_output = Concatenate()([output_rgb, output_optical])
flatten_output = Flatten()(concat_output)

combined_model = Model(inputs=[model_rgb.input, model_optical.input], outputs=flatten_output)
combined_model.trainable = False

In [12]:
# conv_model = Model(inputs = model_rgb.input, outputs = model_rgb.output)
# optical_model = Model(inputs = model_optical.input, outputs = model_optical.output)


# # Rename the layers in each model to ensure uniqueness
# for layer in model_rgb.layers:
#     layer._name = 'rgb_' + layer.name
# for layer in model_optical.layers:
#     layer._name = 'optical_' + layer.name

# output_rgb = model_rgb.output
# output_optical = model_optical.output

# # Define the weights for averaging (you can adjust these values)
# rgb_weight = 0.5
# optical_weight = 0.5

# # Averaging fusion
# weighted_avg_output = Average()([output_rgb * rgb_weight, output_optical * optical_weight])
# flatten_output = Flatten()(weighted_avg_output)

# combined_model = Model(inputs=[model_rgb.input, model_optical.input], outputs=flatten_output)
# combined_model.trainable = False

In [13]:
combined_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 rgb_input_1 (InputLayer)       [(None, 40, 224, 22  0           []                               
                                4, 3)]                                                            
                                                                                                  
 optical_input_2 (InputLayer)   [(None, 40, 224, 22  0           []                               
                                4, 2)]                                                            
                                                                                                  
 rgb_Conv3d_1a_7x7_conv (Conv3D  (None, 20, 112, 112  65856      ['rgb_input_1[0][0]']            
 )                              , 64)                                                       

In [14]:
def euclidean_distance(vects):
    x, y = vects
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))

In [15]:
# os.makedirs('/kaggle/working/output-optical')

In [16]:
# os.makedirs('/kaggle/working/output-conv')

In [17]:
output_dir = '/kaggle/working/output-concatenate'
os.makedirs(output_dir, exist_ok=True)

In [18]:
convolution_dir = '/kaggle/input/complete-conv-dataset/rgb_uniform'
optical_dir = '/kaggle/input/complete-opt-dataset/optical_uniform'

In [19]:
convolution_sequences = sorted(os.listdir(convolution_dir))[1:]
for sequence_name in convolution_sequences:
    sequence_dir = os.path.join(convolution_dir, sequence_name)
    sequence_files = sorted(os.listdir(sequence_dir))

    for file in sequence_files:
        file_path_1 = os.path.join(sequence_dir, file)
        file_path_2 = os.path.join(optical_dir, sequence_name, file)
        print('file_path_1', file_path_1)
        print('file_path_2', file_path_2)
        a1 = np.load(file_path_1)
#         print('a1_shape', a1.shape)
        a2 = np.squeeze(a1)
#         print('a2_shape', a2.shape)
        a3 = np.expand_dims(a2, axis=0)
#         print('a3_shape', a3.shape)
        a4 = np.load(file_path_2)
#         print('a4_shape', a4.shape)
        a5 = np.expand_dims(a4, axis=0)
#         print('a5_shape', a5.shape)
        output = combined_model([a3, a5])
        files = file.split("-")[0]
#         print('files', files)
#         break

        file_name = file.split(".")[0]
        output_path = os.path.join(output_dir, files)
        np.save(output_path, output)

file_path_1 /kaggle/input/complete-conv-dataset/rgb_uniform/BandMarching/v_BandMarching_g01_c01.npy
file_path_2 /kaggle/input/complete-opt-dataset/optical_uniform/BandMarching/v_BandMarching_g01_c01.npy
file_path_1 /kaggle/input/complete-conv-dataset/rgb_uniform/BandMarching/v_BandMarching_g01_c02.npy
file_path_2 /kaggle/input/complete-opt-dataset/optical_uniform/BandMarching/v_BandMarching_g01_c02.npy
file_path_1 /kaggle/input/complete-conv-dataset/rgb_uniform/BandMarching/v_BandMarching_g01_c03.npy
file_path_2 /kaggle/input/complete-opt-dataset/optical_uniform/BandMarching/v_BandMarching_g01_c03.npy
file_path_1 /kaggle/input/complete-conv-dataset/rgb_uniform/BandMarching/v_BandMarching_g01_c04.npy
file_path_2 /kaggle/input/complete-opt-dataset/optical_uniform/BandMarching/v_BandMarching_g01_c04.npy
file_path_1 /kaggle/input/complete-conv-dataset/rgb_uniform/BandMarching/v_BandMarching_g01_c05.npy
file_path_2 /kaggle/input/complete-opt-dataset/optical_uniform/BandMarching/v_BandMarchi